In [1]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\orange pc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\orange
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\orange
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\orange
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import numpy as np
import pandas as pd
import re
import time
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn import tree

In [3]:
tweets = pd.read_csv("Tweets.csv")
pd.DataFrame(tweets)

,sentiment,tweet
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...
...,...,...
74677,Positive,Just realized that the Windows partition of my...
74678,Positive,Just realized that my Mac window partition is ...
74679,Positive,Just realized the windows partition of my Mac ...
74680,Positive,Just realized between the windows partition of...


In [4]:
features = tweets["tweet"]
labels = tweets["sentiment"]

In [5]:

processed_features = []
for sentence in range(0, len(features)):
    processed_feature = re.sub("(@[A-Za-z0-9_]+)", "",
                               str(features[sentence]))
    processed_feature = re.sub(
        'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|' \
        '(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', processed_feature)
 
    processed_feature = re.sub(r"'", '', processed_feature)
 
    processed_feature = re.sub(r'\W', ' ', processed_feature)
 
    processed_feature = re.sub(r'\s+[a-zA-Z1-9]\s+', ' ', processed_feature)
 
    processed_feature = re.sub(r'^[a-zA-Z1-9]\s+', '', processed_feature) 
 
    processed_feature = re.sub(r'\s+[a-zA-Z1-9]$', '', processed_feature) 
 
    processed_feature = re.sub(r'\s+', ' ', processed_feature)
 
    processed_feature = processed_feature.strip()
 
    processed_feature = processed_feature.lower()
    
    processed_features.append(processed_feature)
    

In [6]:

lemmaTokens = []
rawTokens = []
lemmatizer = WordNetLemmatizer()
 
# Tokenizing
for sentence in processed_features:
    tempTokens = sentence.split()
    rawTokens.append(tempTokens)
    lemmatizedTokens = []
    
    # proceed lemmatization
    for singleToken, tag in pos_tag(tempTokens):
        # Preparing for morphological analysis
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
            
        token = lemmatizer.lemmatize(singleToken, pos)
        
        if token not in stopwords.words("english"):
            if token.isdigit() == False:
                lemmatizedTokens.append(token)
        
    # append the lemmatized tokens of sentence in the list
    lemmaTokens.append(lemmatizedTokens)

In [7]:
count = 0
for tok in lemmaTokens:
    if len(tok)==0:
        lemmaTokens.pop(count)
        rawTokens.pop(count)
        labels.pop(count)
        
    count+=1

In [ ]:

words = set()
for documentTokens in lemmaTokens:
    for token in documentTokens:
        words.add(token)
uniqueWords = list(words)
documents = []
for i in range(0,len(lemmaTokens)):
    documents.append(dict.fromkeys(uniqueWords,0))
    
count = 0
for documentTokens in lemmaTokens:
    for token in documentTokens:
        documents[count][token]+=1
    count+=1

def calculateTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        if bowCount == 0:
            print(wordDict)
            print(bow)
        try:
            tfDict[word] = count/float(bowCount)
        except ZeroDivisionError:
            print("error")
    return tfDict
 
TF = []
count = 0
for document in documents:
    TF.append(calculateTF(document,lemmaTokens[count]))
    count += 1

In [ ]:
def calculateIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

IDFs = calculateIDF(documents)

In [ ]:

def calculateTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf
 
TFIDF = []
count = 0
for document in TF:
    TFIDF.append(calculateTFIDF(document,IDFs))
    count += 1

TFIDF = pd.DataFrame(TFIDF)

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(TFIDF, labels, test_size=0.2, random_state = 40)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsRestClassifier
OvR = OneVsRestClassifier(RandomForestClassifier())
OvR.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier())

In [ ]:
OvR.score(X_test,y_test)

0.8872506504770165

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
rf  = RandomForestClassifier()
Sfolds = StratifiedKFold(n_splits=5,shuffle=True)
sc_Srf=cross_val_score(rf,TFIDF,labels,cv=Sfolds)*100
print('Random Forest--> Accuracy for all iteration ', sc_Srf)
ME_Srf = sc_Srf.mean()
print('Random Forest-->Mean Accuracy %.2f' %ME_Srf)

Random Forest--> Accuracy for all iteration  [89.93928881 90.24284475 89.76582827 89.80919341 89.02862099]
Random Forest-->Mean Accuracy 89.76


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
rf  = RandomForestClassifier(n_estimators=50)
Sfolds = StratifiedKFold(n_splits=4,shuffle=True)
sc_Srf=cross_val_score(rf,TFIDF,labels,cv=20)*100
print('Random Forest--> Accuracy for all iteration ', sc_Srf)
ME_Srf = sc_Srf.mean()
print('Random Forest-->Mean Accuracy %.2f' %ME_Srf)

Random Forest--> Accuracy for all iteration  [89.08145581 89.94800693 88.04159445 87.694974   90.46793761 87.34835355
 87.86828423 89.42807626 85.6152513  89.08145581 86.97916667 92.1875
 89.93055556 88.71527778 89.40972222 89.0625     89.0625     89.23611111
 91.66666667 91.31944444]
Random Forest-->Mean Accuracy 89.11


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
rf  = RandomForestClassifier()
Sfolds = StratifiedKFold(n_splits=4,shuffle=True)
sc_Srf=cross_val_score(rf,TFIDF,labels,cv=246)*100
print('Random Forest--> Accuracy for all iteration ', sc_Srf)
ME_Srf = sc_Srf.mean()
print('Random Forest-->Mean Accuracy %.2f' %ME_Srf)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
rf  = RandomForestClassifier(n_estimators=50)
Sfolds = StratifiedKFold(n_splits=246,shuffle=True)
sc_Srf=cross_val_score(rf,TFIDF,labels,cv=20)*100
print('Random Forest--> Accuracy for all iteration ', sc_Srf)
ME_Srf = sc_Srf.mean()
print('Random Forest-->Mean Accuracy %.2f' %ME_Srf)

Random Forest--> Accuracy for all iteration  [88.90814558 89.77469671 88.04159445 89.60138648 89.60138648 86.48180243
 86.65511265 89.60138648 85.96187175 88.3882149  86.97916667 91.49305556
 91.14583333 89.23611111 89.0625     87.84722222 88.02083333 89.75694444
 91.49305556 90.97222222]
Random Forest-->Mean Accuracy 88.95


In [ ]:
from sklearn.linear_model import LogisticRegression  
logic=LogisticRegression()
logic.fit(X_train,y_train)
logic.score(X_test,y_test)

NameError: ignored